In [79]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
from helpers import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [82]:
DATA_TRAIN_PATH = '../Data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Change -999s to median of column (feature) and standardize

In [83]:
tX.shape

(250000, 30)

In [84]:
for col in tX.T:
    med = np.median(col[col != -999])
    col[col == -999] = med
    
#keep model index column seperate
model_index = tX[:, 22]

#remove model index column from tx
tX = np.delete(tX, 22, 1)
    
#Standardize
#tX, mean_x, std_x = standardize(tX)
tX2 = np.hstack((np.ones((tX.shape[0],1)), tX))

print(tX2.shape)

(250000, 30)


In [85]:
tX2.shape

(250000, 30)

## Build Polyonmial Matrix

In [86]:
degree = 2
#polyX = np.full((tX.shape[0], tX.shape[1] * (degree+1)), 0.0)
polyX = list()
for row in tX2:
    polyX.append(build_poly2(row, degree))

PolyXNP = np.array(polyX)
PolyXNP2 = PolyXNP.reshape((PolyXNP.shape[0], PolyXNP.shape[2]))


In [87]:
PolyXNP2.shape

(250000, 59)

## Training with 4 models:

In [88]:
Fused = np.c_[y, PolyXNP2]

In [89]:
from ml_functions import *

#First we build a polynomial model to represent the features
#phi = build_poly(tX, 1)

Fused_0 = Fused[model_index == 0.0]
Fused_1 = Fused[model_index == 1.0]
Fused_2 = Fused[model_index == 2.0]
Fused_3 = Fused[model_index == 3.0]
#TODO remove feature
gamma = 0.05
lambda_ = 0.5
max_iters = 100

loss1, w1 = pen_logistic_regression_test(Fused_0[:,0], Fused_0[:,1:Fused_0.shape[1]], lambda_, gamma, max_iters)
loss2, w2 = pen_logistic_regression_test(Fused_1[:,0], Fused_1[:,1:Fused_1.shape[1]], lambda_, gamma, max_iters)
loss3, w3 = pen_logistic_regression_test(Fused_2[:,0], Fused_2[:,1:Fused_2.shape[1]], lambda_, gamma, max_iters)
loss4, w4 = pen_logistic_regression_test(Fused_3[:,0], Fused_3[:,1:Fused_3.shape[1]], lambda_, gamma, max_iters)

#weights = ridge_regression(y, tX, 0.01)

weights = (w1, w2, w3, w4)

#w = logistic_regression(y,tX,gamma,max_iters)

In [90]:
w2.shape

(59, 1)

In [91]:
weights2 = (w1.reshape((w1.shape[0],)), w2.reshape((w2.shape[0],)),w3.reshape((w3.shape[0],)), w4.reshape((w4.shape[0],)))


In [92]:
w1.shape

(59, 1)

## Generate predictions and save ouput in csv format for submission:

In [93]:
DATA_TEST_PATH = '../Data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

### Prepare Test Data

In [94]:
for col in tX_test.T:
    med = np.median(col[col != -999])
    col[col == -999] = med

#keep model index column seperate
model_index2 = tX_test[:, 22]

#remove model index column from tx
tX_test = np.delete(tX_test, 22, 1)

#Standardize
#tX_test, mean_x, std_x = standardize(tX_test)
tX_test2 = np.hstack((np.ones((tX_test.shape[0],1)), tX_test))

#Use list to calculate polynomial faster
polyXTest = list()
for row in tX_test2:
    polyXTest.append(build_poly2(row, degree))

PolyXNPTest = np.array(polyXTest)
PolyXNPTest2 = PolyXNPTest.reshape((PolyXNPTest.shape[0], PolyXNPTest.shape[2]))

PolyXNPTest2.shape

(568238, 59)

In [95]:
OUTPUT_PATH = '../Data/resultLogisticPenalized6.csv' 
# TODO: fill in desired name of output file for submission
y_pred = predict_labels3(weights2, PolyXNPTest2, degree, model_index2)
print(y_pred.shape)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

[ 0.  0.  0. ...,  0.  0.  0.]
(568238,)


In [96]:
print(y_pred)

[-1. -1. -1. ..., -1. -1. -1.]
